In [1]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 40.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=821620adf9aec9b6ef6944e89173a9518bcc769911c406499b11a199403e3611
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('kmeans').getOrCreate()

In [5]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
file_path = '/content/drive/MyDrive/'
data=spark.read.csv(file_path +'seeds_dataset.csv',inferSchema=True,header=True)

In [10]:
data.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [11]:
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [12]:
data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [13]:
from pyspark.ml.clustering import KMeans

In [14]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=data.columns,outputCol="Independent Features")

In [15]:
finaldata=featureassembler.transform(data)

In [16]:
finaldata.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- Independent Features: vector (nullable = true)



In [17]:
from pyspark.ml.feature import StandardScaler

In [18]:
sc=StandardScaler(inputCol='Independent Features',outputCol='Scaled Features')

In [19]:
scaled=sc.fit(finaldata)

In [20]:
finaldata=scaled.transform(finaldata)

In [21]:
finaldata.head() # we get scaled Features column

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, Independent Features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), Scaled Features=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))

In [24]:
kmeans=KMeans(featuresCol='Scaled Features',k=3)

In [25]:
model=kmeans.fit(finaldata)

In [26]:
clustercenter=model.clusterCenters()

In [27]:
clustercenter  # 3 centers as we gave k=3

[array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,  8.55443412,
         1.81649011, 10.32998598]),
 array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,  9.748067  ,
         2.39849968, 12.2661748 ]),
 array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,  7.50395937,
         3.27184732, 10.42126018])]

In [28]:
model.transform(finaldata)

DataFrame[area: double, perimeter: double, compactness: double, length_of_kernel: double, width_of_kernel: double, asymmetry_coefficient: double, length_of_groove: double, Independent Features: vector, Scaled Features: vector, prediction: int]

In [29]:
model.transform(finaldata).select('prediction').show()  # gives groups of 0,1,2

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows



In [31]:
model.transform(finaldata).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   70|
|         2|   65|
|         0|   75|
+----------+-----+

